In [1]:
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data
from datetime import datetime
#载入数据
mnist = input_data.read_data_sets("/root/code/MNIST_data/",one_hot=True)
#载入图片是 28*28
n_inputs = 28 #输入一共28行
max_time = 28 #一共28行
lstm_size = 100#隐层单元 block
n_classes = 10 #10个分类
batch_size = 100 #每批次50个样本
n_batch = mnist.train.num_examples//batch_size #计算一共有多少批次

#这里的none 表示第一维度可以为任意长度
x = tf.placeholder(tf.float32,[None,784],name='x')
#正确的标签
y = tf.placeholder(tf.float32,[None,10],name='y')

#初始化权重
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
#初始化偏置
biases =tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义run网络
def RUN(X,weights,biases,re_name):
    # inputs = [batch_size,max_time,n_inputs]
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL
    """
    tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True): 
    n_hidden表示神经元的个数，
    forget_bias就是LSTM们的忘记系数，默认为1，如果等于1，就是不会忘记任何信息。如果等于0，就都忘记。
    state_is_tuple默认就是True，官方建议用True，就是表示返回的状态用一个元祖表示。
    这个里面存在一个状态初始化函数，就是zero_state（batch_size，dtype）两个参数。
    batch_size就是输入样本批次的数目，
    dtype就是数据类型。
    """
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    #final_state[0]是cell state
    #final_state[1]是hidden state
    """
    tf.nn.dynamic_rnn(cell,inputs,sequence_length=None,initial_state=None,dtype=None,parallel_iterations=None,swap_memory=False,
    time_major=False,scope=None )
    cell: RNNCell的一个实例.

inputs: RNN输入.

如果time_major == False(默认), 则是一个shape为[batch_size, max_time, input_size]的Tensor,或者这些元素的嵌套元组。

如果time_major == True,则是一个shape为[max_time, batch_size, input_size]的Tensor,或这些元素的嵌套元组。

sequence_length: (可选）大小为[batch_size],数据的类型是int32/int64向量。如果当前时间步的index超过该序列的实际长度时，
则该时间步不进行计算，RNN的state复制上一个时间步的，同时该时间步的输出全部为零。

initial_state: (可选)RNN的初始state(状态)。如果cell.state_size(一层的RNNCell)是一个整数，
那么它必须是一个具有适当类型和形状的张量[batch_size，cell.state_size]。如果cell.state_size是一个元组(多层的RNNCell,如MultiRNNCell)，
那么它应该是一个张量元组，每个元素的形状为[batch_size，s] for s in cell.state_size。

time_major: inputs 和outputs 张量的形状格式。如果为True，则这些张量都应该是（都会是）[max_time, batch_size, depth]。
如果为false，则这些张量都应该是（都会是）[batch_size，max_time, depth]。time_major=true说明输入和输出tensor的第一维是max_time。
否则为batch_size。

使用time_major =True更有效,因为它避免了RNN计算开始和结束时的转置.但是,大多数TensorFlow数据都是batch-major,因此默认情况下,
此函数接受输入并以batch-major形式发出输出.

返回值：

一对(outputs, state),其中：

outputs： RNN输出Tensor.

如果time_major == False(默认),这将是shape为[batch_size, max_time, cell.output_size]的Tensor.

如果time_major == True,这将是shape为[max_time, batch_size, cell.output_size]的Tensor.

state： 最终的状态.

一般情况下state的形状为 [batch_size, cell.output_size ]

如果cell是LSTMCells,则state将是包含每个单元格的LSTMStateTuple的元组，state的形状为[2，batch_size, cell.output_size ]

    """
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases,name=re_name)
    return results

#计算RUN的返回结果
result = "result"
prediction = RUN(x,weights,biases,result)
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels =y))
#使用Adamoptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction= tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#求准确率

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

saver = tf.train.Saver() #定义saver


#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    start = datetime.now()
    for epoch in range(5001):
#         for batch in range(n_batch):
        batch_xs,batch_ys =mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter"+str(epoch)+",testing accuracy="+ str(acc))
    end = datetime.now()
    latency = (end - start).total_seconds()
    print('训练5000步总时长为 '+str(latency)+' 秒 ')
    saver.save(sess, '/tmp/tf_rnn_1_100/model.ckpt') #模型储存位置




/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /root/code/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /root/code/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /root/code/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /root/code/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Pleas

Iter141,testing accuracy=0.2658
Iter142,testing accuracy=0.266
Iter143,testing accuracy=0.2659
Iter144,testing accuracy=0.2654
Iter145,testing accuracy=0.2661
Iter146,testing accuracy=0.2667
Iter147,testing accuracy=0.2677
Iter148,testing accuracy=0.2687
Iter149,testing accuracy=0.2701
Iter150,testing accuracy=0.2714
Iter151,testing accuracy=0.2724
Iter152,testing accuracy=0.2732
Iter153,testing accuracy=0.2739
Iter154,testing accuracy=0.2745
Iter155,testing accuracy=0.2754
Iter156,testing accuracy=0.2763
Iter157,testing accuracy=0.2773
Iter158,testing accuracy=0.2783
Iter159,testing accuracy=0.2793
Iter160,testing accuracy=0.2794
Iter161,testing accuracy=0.2803
Iter162,testing accuracy=0.2812
Iter163,testing accuracy=0.2814
Iter164,testing accuracy=0.2826
Iter165,testing accuracy=0.2829
Iter166,testing accuracy=0.284
Iter167,testing accuracy=0.2842
Iter168,testing accuracy=0.285
Iter169,testing accuracy=0.2853
Iter170,testing accuracy=0.2861
Iter171,testing accuracy=0.2867
Iter172,tes

Iter398,testing accuracy=0.5237
Iter399,testing accuracy=0.5245
Iter400,testing accuracy=0.524
Iter401,testing accuracy=0.5235
Iter402,testing accuracy=0.5222
Iter403,testing accuracy=0.521
Iter404,testing accuracy=0.5206
Iter405,testing accuracy=0.52
Iter406,testing accuracy=0.5204
Iter407,testing accuracy=0.5224
Iter408,testing accuracy=0.5233
Iter409,testing accuracy=0.5241
Iter410,testing accuracy=0.5242
Iter411,testing accuracy=0.5243
Iter412,testing accuracy=0.5244
Iter413,testing accuracy=0.5247
Iter414,testing accuracy=0.5235
Iter415,testing accuracy=0.5244
Iter416,testing accuracy=0.5256
Iter417,testing accuracy=0.5252
Iter418,testing accuracy=0.5255
Iter419,testing accuracy=0.5257
Iter420,testing accuracy=0.5253
Iter421,testing accuracy=0.5271
Iter422,testing accuracy=0.527
Iter423,testing accuracy=0.5266
Iter424,testing accuracy=0.5281
Iter425,testing accuracy=0.5282
Iter426,testing accuracy=0.5262
Iter427,testing accuracy=0.5263
Iter428,testing accuracy=0.5262
Iter429,testi

Iter655,testing accuracy=0.568
Iter656,testing accuracy=0.5689
Iter657,testing accuracy=0.5706
Iter658,testing accuracy=0.5719
Iter659,testing accuracy=0.5726
Iter660,testing accuracy=0.5727
Iter661,testing accuracy=0.5729
Iter662,testing accuracy=0.5725
Iter663,testing accuracy=0.5722
Iter664,testing accuracy=0.5738
Iter665,testing accuracy=0.5743
Iter666,testing accuracy=0.5742
Iter667,testing accuracy=0.5766
Iter668,testing accuracy=0.5773
Iter669,testing accuracy=0.5776
Iter670,testing accuracy=0.5787
Iter671,testing accuracy=0.5803
Iter672,testing accuracy=0.58
Iter673,testing accuracy=0.5816
Iter674,testing accuracy=0.5814
Iter675,testing accuracy=0.5818
Iter676,testing accuracy=0.5816
Iter677,testing accuracy=0.5817
Iter678,testing accuracy=0.5827
Iter679,testing accuracy=0.5838
Iter680,testing accuracy=0.5861
Iter681,testing accuracy=0.588
Iter682,testing accuracy=0.5889
Iter683,testing accuracy=0.5896
Iter684,testing accuracy=0.5898
Iter685,testing accuracy=0.5904
Iter686,test

...

Iter3912,testing accuracy=0.9155
Iter3913,testing accuracy=0.915
Iter3914,testing accuracy=0.9146
Iter3915,testing accuracy=0.9126
Iter3916,testing accuracy=0.9131
Iter3917,testing accuracy=0.9132
Iter3918,testing accuracy=0.9138
Iter3919,testing accuracy=0.9144
Iter3920,testing accuracy=0.9137
Iter3921,testing accuracy=0.9151
Iter3922,testing accuracy=0.9156
Iter3923,testing accuracy=0.9158
Iter3924,testing accuracy=0.9162
Iter3925,testing accuracy=0.9164
Iter3926,testing accuracy=0.9161
Iter3927,testing accuracy=0.9159
Iter3928,testing accuracy=0.9152
Iter3929,testing accuracy=0.9154
Iter3930,testing accuracy=0.9149
Iter3931,testing accuracy=0.9144
Iter3932,testing accuracy=0.9146
Iter3933,testing accuracy=0.9145
Iter3934,testing accuracy=0.9139
Iter3935,testing accuracy=0.9137
Iter3936,testing accuracy=0.9126
Iter3937,testing accuracy=0.9134
Iter3938,testing accuracy=0.9129
Iter3939,testing accuracy=0.9136
Iter3940,testing accuracy=0.9139
Iter3941,testing accuracy=0.9141
Iter3942,te

Iter4161,testing accuracy=0.9231
Iter4162,testing accuracy=0.9213
Iter4163,testing accuracy=0.9204
Iter4164,testing accuracy=0.9202
Iter4165,testing accuracy=0.919
Iter4166,testing accuracy=0.915
Iter4167,testing accuracy=0.9114
Iter4168,testing accuracy=0.9094
Iter4169,testing accuracy=0.9096
Iter4170,testing accuracy=0.9105
Iter4171,testing accuracy=0.9122
Iter4172,testing accuracy=0.9159
Iter4173,testing accuracy=0.9186
Iter4174,testing accuracy=0.9205
Iter4175,testing accuracy=0.9216
Iter4176,testing accuracy=0.9195
Iter4177,testing accuracy=0.9158
Iter4178,testing accuracy=0.9101
Iter4179,testing accuracy=0.9052
Iter4180,testing accuracy=0.9036
Iter4181,testing accuracy=0.906
Iter4182,testing accuracy=0.9093
Iter4183,testing accuracy=0.9145
Iter4184,testing accuracy=0.9185
Iter4185,testing accuracy=0.9173
Iter4186,testing accuracy=0.9138
Iter4187,testing accuracy=0.9087
Iter4188,testing accuracy=0.9037
Iter4189,testing accuracy=0.9014
Iter4190,testing accuracy=0.8976
Iter4191,test

Iter4411,testing accuracy=0.925
Iter4412,testing accuracy=0.9238
Iter4413,testing accuracy=0.9231
Iter4414,testing accuracy=0.9222
Iter4415,testing accuracy=0.9184
Iter4416,testing accuracy=0.917
Iter4417,testing accuracy=0.9194
Iter4418,testing accuracy=0.9203
Iter4419,testing accuracy=0.9215
Iter4420,testing accuracy=0.9216
Iter4421,testing accuracy=0.9208
Iter4422,testing accuracy=0.9208
Iter4423,testing accuracy=0.9216
Iter4424,testing accuracy=0.922
Iter4425,testing accuracy=0.9229
Iter4426,testing accuracy=0.9234
Iter4427,testing accuracy=0.9237
Iter4428,testing accuracy=0.9246
Iter4429,testing accuracy=0.9255
Iter4430,testing accuracy=0.9263
Iter4431,testing accuracy=0.9262
Iter4432,testing accuracy=0.926
Iter4433,testing accuracy=0.9254
Iter4434,testing accuracy=0.9254
Iter4435,testing accuracy=0.9243
Iter4436,testing accuracy=0.9233
Iter4437,testing accuracy=0.921
Iter4438,testing accuracy=0.9171
Iter4439,testing accuracy=0.9127
Iter4440,testing accuracy=0.9124
Iter4441,testin

Iter4661,testing accuracy=0.9108
Iter4662,testing accuracy=0.9133
Iter4663,testing accuracy=0.9153
Iter4664,testing accuracy=0.9195
Iter4665,testing accuracy=0.9216
Iter4666,testing accuracy=0.9235
Iter4667,testing accuracy=0.9261
Iter4668,testing accuracy=0.9266
Iter4669,testing accuracy=0.9265
Iter4670,testing accuracy=0.9273
Iter4671,testing accuracy=0.9263
Iter4672,testing accuracy=0.9252
Iter4673,testing accuracy=0.9237
Iter4674,testing accuracy=0.9238
Iter4675,testing accuracy=0.9237
Iter4676,testing accuracy=0.9231
Iter4677,testing accuracy=0.9231
Iter4678,testing accuracy=0.924
Iter4679,testing accuracy=0.9242
Iter4680,testing accuracy=0.925
Iter4681,testing accuracy=0.9252
Iter4682,testing accuracy=0.9261
Iter4683,testing accuracy=0.9275
Iter4684,testing accuracy=0.927
Iter4685,testing accuracy=0.9249
Iter4686,testing accuracy=0.9241
Iter4687,testing accuracy=0.9245
Iter4688,testing accuracy=0.9243
Iter4689,testing accuracy=0.9237
Iter4690,testing accuracy=0.9243
Iter4691,test

Iter4911,testing accuracy=0.9266
Iter4912,testing accuracy=0.9262
Iter4913,testing accuracy=0.9262
Iter4914,testing accuracy=0.9249
Iter4915,testing accuracy=0.9266
Iter4916,testing accuracy=0.9269
Iter4917,testing accuracy=0.9276
Iter4918,testing accuracy=0.9287
Iter4919,testing accuracy=0.9291
Iter4920,testing accuracy=0.9297
Iter4921,testing accuracy=0.9296
Iter4922,testing accuracy=0.9296
Iter4923,testing accuracy=0.9292
Iter4924,testing accuracy=0.93
Iter4925,testing accuracy=0.9302
Iter4926,testing accuracy=0.9301
Iter4927,testing accuracy=0.9298
Iter4928,testing accuracy=0.9299
Iter4929,testing accuracy=0.9291
Iter4930,testing accuracy=0.9288
Iter4931,testing accuracy=0.9283
Iter4932,testing accuracy=0.9298
Iter4933,testing accuracy=0.9307
Iter4934,testing accuracy=0.9307
Iter4935,testing accuracy=0.9282
Iter4936,testing accuracy=0.925
Iter4937,testing accuracy=0.9225
Iter4938,testing accuracy=0.9189
Iter4939,testing accuracy=0.9147
Iter4940,testing accuracy=0.9112
Iter4941,test

此处说明for循环嵌套 sess训练了 更多的次数 可以去除 训练效果是一样的